# 세팅

In [ ]:
import os

# 1. 드라이브 마운트 (이미 되어 있다면 패스)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import zipfile
import json
import cv2
import numpy as np
from tqdm import tqdm

# 2. 경로 설정
zip_path = '/content/drive/MyDrive/LikeLion_CV/b. Segmentation_Team/Preprocessed_Zip/final_data.zip'
extract_path = '/' # content/split_dataset 구조 유지

# 3. 압축 해제
if os.path.exists(zip_path):
    print("📦 1. 데이터를 작업대(/content)로 압축 해제 중...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        file_list = z.namelist()
        for file in tqdm(file_list, desc="압축 해제", unit="file"):
            z.extract(file, extract_path)
    print("✅ 압축 해제 완료!")
else:
    print("❌ 파일을 찾을 수 없습니다!")

# 4. JSON -> PNG 마스크 사전 생성 (병목 제거 핵심)
def pre_generate_masks(subset):
    base_path = f'/content/split_dataset/{subset}'
    json_dir = os.path.join(base_path, 'label')
    img_dir = os.path.join(base_path, 'images')
    mask_save_dir = os.path.join(base_path, 'masks_png')
    os.makedirs(mask_save_dir, exist_ok=True)

    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]
    print(f"🖼️ {subset} 데이터 마스크 생성 중 (크기 맞춤 모드)...")

    for j_file in tqdm(json_files):
        # 1. 대응하는 이미지 파일을 읽어서 실제 크기 확인
        # 확장자가 다를 수 있으므로 체크 (png, jpg, jpeg)
        img_base = os.path.splitext(j_file)[0]
        img_name = None
        for ext in ['.png', '.jpg', '.jpeg', '.JPG']:
            if os.path.exists(os.path.join(img_dir, img_base + ext)):
                img_name = img_base + ext
                break

        if img_name is None: continue # 이미지가 없으면 패스

        # 이미지 헤더만 읽어서 크기 확인 (속도 향상)
        tmp_img = cv2.imread(os.path.join(img_dir, img_name))
        h, w = tmp_img.shape[:2]

        # 2. 확인된 크기로 마스크 도화지 생성
        with open(os.path.join(json_dir, j_file), 'r', encoding='utf-8') as f:
            data = json.load(f)

        mask = np.zeros((h, w), dtype=np.uint8)

        # Outline (3)
        sw = data.get('swelling')
        if sw and sw.get('battery_outline'):
            pts = np.array(sw['battery_outline']).reshape(-1, 2).astype(np.int32)
            cv2.fillPoly(mask, [pts], color=3)

        # Defects (1, 2)
        defects = data.get('defects')
        if defects:
            for d in defects:
                if d.get('points'):
                    pts = np.array(d['points']).reshape(-1, 2).astype(np.int32)
                    cls = 1 if d['name'] == 'Pollution' else 2
                    cv2.fillPoly(mask, [pts], color=cls)

        cv2.imwrite(os.path.join(mask_save_dir, j_file.replace('.json', '.png')), mask)

pre_generate_masks('train')
pre_generate_masks('val')
print("🚀 모든 준비가 끝났습니다.")

📦 1. 데이터를 작업대(/content)로 압축 해제 중...


압축 해제: 100%|██████████| 44501/44501 [06:22<00:00, 116.40file/s] 


✅ 압축 해제 완료!
🖼️ train 데이터 마스크 생성 중 (크기 맞춤 모드)...


100%|██████████| 17797/17797 [13:26<00:00, 22.07it/s]


🖼️ val 데이터 마스크 생성 중 (크기 맞춤 모드)...


100%|██████████| 4450/4450 [03:21<00:00, 22.13it/s]

🚀 모든 준비가 끝났습니다.


# 실행

In [ ]:
!python "/content/drive/MyDrive/LikeLion_CV/b. Segmentation_Team/attention_unet/main.py"

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100% 21.1M/21.1M [00:00<00:00, 158MB/s]

🚀 Epoch 1/30

Training  : 100% 557/557 [14:32<00:00,  1.57s/it]
📊 training   | Loss: 25.4012 | mIoU: 0.3244
   └─ IoU [BG: 0.4373 | Poll: 0.2454 | Dam: 0.0000 | Out: 0.6149]

Validation: 100% 140/140 [03:21<00:00,  1.44s/it]
📊 validation | Loss: 24.1396 | mIoU: 0.4113
   └─ IoU [BG: 0.6747 | Poll: 0.2758 | Dam: 0.0000 | Out: 0.6901]


🚀 Epoch 2/30

Training  : 100% 557/557 [13:55<00:00,  1.50s/it]
📊 training   | Loss: 20.9897 | mIoU: 0.4447
   └─ IoU [BG: 0.5122 | Poll: 0.4583 | Dam: 0.0000 | Out: 0.8083]

Validation: 100% 140/140 [03:14<00:00,  1.39s/it]
📊 validation | Loss: 22.7261 | mIoU: 0.4064
   └─ IoU [BG: 0.4878 | Poll: 0.3011 | Dam: 0.0000 | Out: 0.8319]


🚀 Epoch 3/30

Training  : 100% 557/557 [13:43<00:00,  1.48s/it]
📊 training   | Loss: 19.7969 | mIoU: 0.4798
   └─ IoU [BG: 0.5302 | 

# 불량 배터리 검출 웹 앱

In [11]:
# Streamlit 및 시각화 도구 설치
!pip install streamlit albumentations pyngrok

# Cloudflare Tunnel 설치 (가장 안정적인 외부 접속 도구)
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2026-01-22 02:18:14--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.1.1/cloudflared-linux-amd64.deb [following]
--2026-01-22 02:18:15--  https://github.com/cloudflare/cloudflared/releases/download/2026.1.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/12aae843-3db0-4414-9b56-e2a442db3e76?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-22T03%3A07%3A05Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b

In [12]:
# model_v3.py

import torch
import torch.nn as nn
import torchvision.models as models

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.conv(x)

class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpConv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.up(x)

class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, n_coefficients):
        super(AttentionBlock, self).__init__()
        self.W_gate = nn.Sequential(
            nn.Conv2d(F_g, n_coefficients, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(n_coefficients)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, n_coefficients, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(n_coefficients)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(n_coefficients, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, gate, skip_connection):
        g1 = self.W_gate(gate)
        x1 = self.W_x(skip_connection)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return skip_connection * psi

# 🔥 ASPP 모듈 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, rates=[1, 6, 12, 18]):
        super(ASPP, self).__init__()
        self.stages = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, bias=False),
                          nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True))
        ] + [
            nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, padding=r, dilation=r, bias=False),
                          nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True))
            for r in rates[1:]
        ])
        self.bottleneck = nn.Sequential(
            nn.Conv2d(out_channels * len(rates), out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)
        )
    def forward(self, x):
        res = torch.cat([stage(x) for stage in self.stages], dim=1)
        return self.bottleneck(res)

class AttentionUNet_MobileNetV3(nn.Module):
    def __init__(self, output_ch=4, pretrained=True):
        super(AttentionUNet_MobileNetV3, self).__init__()
        mobilenet = models.mobilenet_v3_large(weights='DEFAULT' if pretrained else None).features

        self.enc1 = mobilenet[0:2]    # 16 ch
        self.enc2 = mobilenet[2:4]    # 24 ch
        self.enc3 = mobilenet[4:7]    # 40 ch
        self.enc4 = mobilenet[7:13]   # 112 ch
        self.enc5 = mobilenet[13:16]  # 160 ch

        # 🔥 Bottleneck에 ASPP 배치
        self.aspp = ASPP(160, 160)

        self.Up5 = UpConv(160, 112)
        self.Att5 = AttentionBlock(F_g=112, F_l=112, n_coefficients=56)
        self.UpConv5 = ConvBlock(224, 112)

        self.Up4 = UpConv(112, 40)
        self.Att4 = AttentionBlock(F_g=40, F_l=40, n_coefficients=20)
        self.UpConv4 = ConvBlock(80, 40)

        self.Up3 = UpConv(40, 24)
        self.Att3 = AttentionBlock(F_g=24, F_l=24, n_coefficients=12)
        self.UpConv3 = ConvBlock(48, 24)

        self.Up2 = UpConv(24, 16)
        self.Att2 = AttentionBlock(F_g=16, F_l=16, n_coefficients=8)
        self.UpConv2 = ConvBlock(32, 16)

        self.Up_final = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.Conv_1x1 = nn.Conv2d(16, output_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        e1 = self.enc1(x); e2 = self.enc2(e1); e3 = self.enc3(e2); e4 = self.enc4(e3); e5 = self.enc5(e4)

        # 🔥 ASPP 적용
        e5 = self.aspp(e5)

        d5 = self.Up5(e5)
        s4 = self.Att5(gate=d5, skip_connection=e4)
        d5 = torch.cat((s4, d5), dim=1); d5 = self.UpConv5(d5)

        d4 = self.Up4(d5); s3 = self.Att4(gate=d4, skip_connection=e3)
        d4 = torch.cat((s3, d4), dim=1); d4 = self.UpConv4(d4)

        d3 = self.Up3(d4); s2 = self.Att3(gate=d3, skip_connection=e2)
        d3 = torch.cat((s2, d3), dim=1); d3 = self.UpConv3(d3)

        d2 = self.Up2(d3); s1 = self.Att2(gate=d2, skip_connection=e1)
        d2 = torch.cat((s1, d2), dim=1); d2 = self.UpConv2(d2)

        out = self.Up_final(d2); out = self.Conv_1x1(out)
        return out

In [13]:
%%writefile app.py
import streamlit as st
import torch
import cv2
import numpy as np
from model_v3 import AttentionUNet_MobileNetV3  # 우리가 만든 모델 파일
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# 1. 모델 로드 설정
@st.cache_resource # 모델을 한 번만 로드하도록 캐싱
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # ASPP가 포함된 최종 모델 구조 선언
    model = AttentionUNet_MobileNetV3(output_ch=4)
    # 저장했던 best_model.pth 경로 지정
    state_dict = torch.load('/content/drive/MyDrive/LikeLion_CV/b. Segmentation_Team/results_exp08_09/best_model.pth', map_location=device)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model, device

# 2. 전처리 함수
def preprocess(image):
    transform = A.Compose([
        A.Resize(512, 512),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    return transform(image=image)['image'].unsqueeze(0)

# 3. Streamlit UI 구성
st.set_page_config(page_title="배터리 결함 탐지 AI", layout="wide")
st.title("🔋 배터리 결함 탐지 실시간 데모")
st.write("이미지를 업로드하면 AI가 오염(Pollution)과 손상(Damaged) 부위를 분석합니다.")

model, device = load_model()

uploaded_file = st.file_uploader("배터리 이미지 업로드...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    # 이미지 읽기
    image = Image.open(uploaded_file).convert("RGB")
    img_np = np.array(image)

    col1, col2 = st.columns(2)
    with col1:
        st.image(image, caption="원본 이미지", use_column_width=True)

    # 추론 진행
    with st.spinner('AI 분석 중...'):
        input_tensor = preprocess(img_np).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 결과 시각화 (오버레이)
    # 클래스: 0:배경, 1:Pollution(녹색), 2:Damaged(적색), 3:Outline(청색)
    color_map = {1: [0, 255, 0], 2: [255, 0, 0], 3: [0, 0, 255]}
    overlay = img_np.copy()
    overlay = cv2.resize(overlay, (512, 512))

    for cls, color in color_map.items():
        mask = (pred == cls).astype(np.uint8)
        if mask.any():
            colored_mask = np.zeros_like(overlay)
            colored_mask[mask == 1] = color
            overlay = cv2.addWeighted(overlay, 1, colored_mask, 0.5, 0)

    with col2:
        st.image(overlay, caption="결함 분석 결과", use_column_width=True)

    # 결과 요약
    st.subheader("📌 분석 결과")
    poll_detected = "검출" if 1 in pred else "정상"
    dam_detected = "검출" if 2 in pred else "정상"
    st.write(f"- **오염(Pollution):** {poll_detected}")
    st.write(f"- **손상(Damaged):** {dam_detected}")

Writing app.py


In [14]:
import subprocess
# 1. 스트림릿 서버 백그라운드 실행
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# 2. 클라우드플레어 터널 실행
!cloudflared tunnel --url http://localhost:8501 --protocol http2

2026-01-22T02:21:06Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-22T02:21:06Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-22T02:21:11Z INF +--------------------------------------------------------------------------------------------+
2026-01-22T02:21:11Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-22T02:21:11Z INF |  https://reflects-alberta-pharmacies-cradle.trycloudfl